In [1]:
import tensorflow as tf
from tensorflow import keras
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers

In [3]:
batch_size=16
epochs=20
input_shape = (20,21,3)
classes = 9

In [20]:
x,y=[],[]
with h5py.File('dynamic_gestures2.hdf5', 'a') as f:
    print(f.filename)
    i=0
    for _,gesture in enumerate(list(f['/'])):
        if not "swipe" in gesture:
            continue
        print(i,gesture)
        data = list(f['/'+gesture])
        x.extend(data)
        y.extend([i]*len(data))
        i+=1
        if gesture == "swipe_right":
            print(i,"swipe_left")
        elif gesture == "swipe_up":
            print(i,"swipe_down")
        data = list(np.flip(np.array(list(f['/'+gesture])), axis=1))
        x.extend(data)
        y.extend([i]*len(data))
        i+=1

x=np.array(x)
y=np.array(y) 
y=tf.keras.utils.to_categorical(y,classes)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

dynamic_gestures2.hdf5
0 swipe_right
1 swipe_left
2 swipe_up
3 swipe_down
(669, 20, 21, 3) (75, 20, 21, 3) (669, 9) (75, 9)


In [21]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.Flatten()(x)
print(x.shape)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()


(None, 1280)
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 20, 63)            0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 20, 64)   

In [22]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=0.1)
model.evaluate(x_test,y_test)

Epoch 1/20
38/38 [==============================] - 1s 10ms/step - loss: 1.5587 - accuracy: 0.4868 - val_loss: 0.0594 - val_accuracy: 0.9851
Epoch 2/20
38/38 [==============================] - 0s 5ms/step - loss: 0.0336 - accuracy: 0.9930 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/20
38/38 [==============================] - 0s 5ms/step - loss: 0.0093 - accuracy: 0.9979 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 4/20
38/38 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 5.4565e-04 - val_accuracy: 1.0000
Epoch 5/20
38/38 [==============================] - 0s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 1.5033e-04 - val_accuracy: 1.0000
Epoch 6/20
38/38 [==============================] - 0s 5ms/step - loss: 2.7062e-04 - accuracy: 1.0000 - val_loss: 1.1088e-04 - val_accuracy: 1.0000
Epoch 7/20
38/38 [==============================] - 0s 5ms/step - loss: 1.8521e-04 - accuracy: 1.0000 - val_loss: 9.7065e-05 - val_accuracy: 1.

[0.00028134274180047214, 1.0]

In [23]:
model.save("test_model_2", save_format = "tf")

INFO:tensorflow:Assets written to: test_model_2\assets


In [4]:
!pip --version

pip 19.1.1 from C:\ProgramData\Anaconda3\lib\site-packages\pip (python 3.7)

